In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [8]:
def datapreparation(filepath):
    # Load the dataset
    df = pd.read_csv(filepath)
    df.drop(["custmorerID"], axis=1, inplace=True)
    
    df.TotalCharges = df.TotalCharges.replace(" ", np.nan)
    df.TotalCharges.fillna(0, inplace=True)
    df.TotalCharges = df.TotalCharges.astype(float)
    
    cols1 = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
    for col in cols1:
        df[col] = df[col].apply(lambda x: 0 if x == 'No' else 1)
        df.gender = df.gender.apply(lambda x: 0 if x == 'Male' else 1)
        df.MultipleLines = df.MultipleLines.map({'No phone service' :0, 'No' :0, 'Yes' :1})
        
        cols2 = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
        for col in cols2:
            df[col] = df[col].map({'No internet service' :0, 'No' :0, 'Yes' :1})
            
            df = pd.get_deummies(data=df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)
            
            return df

In [9]:
df = datapreparation(filepath = "https://github.com/amitkumargope/AI-Engineer-Roadmap/blob/e18bd79ccff93259171ff20495035ef87ebc180c/Churn%20Prediction_Business_Analytics/Telco-Customer-Churn.csv")
df.head()

HTTPError: HTTP Error 404: Not Found